In [33]:
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin
from category_encoders import TargetEncoder
from sklearn.preprocessing import LabelEncoder, FunctionTransformer

import pandas as pd

In [8]:
# read in data 

train = pd.read_csv('../project_data/train_data.csv')
train.head()

/var/folders/nj/p20z0h9x3ssf6ll0_cdhh2yc0000gn/T/ipykernel_14007/1611774703.py:3: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv('../project_data/train_data.csv')


,Accident Date,Age at Injury,Alternative Dispute Resolution,Assembly Date,Attorney/Representative,Average Weekly Wage,Birth Year,C-2 Date,C-3 Date,Carrier Name,...,WCIO Cause of Injury Code,WCIO Cause of Injury Description,WCIO Nature of Injury Code,WCIO Nature of Injury Description,WCIO Part Of Body Code,WCIO Part Of Body Description,Zip Code,Agreement Reached,WCB Decision,Number of Dependents
0,2019-12-30,31.0,N,2020-01-01,N,0.00,1988.0,2019-12-31,NaN,NEW HAMPSHIRE INSURANCE CO,...,27.0,FROM LIQUID OR GREASE SPILLS,10.0,CONTUSION,62.0,BUTTOCKS,13662,0.0,Not Work Related,1.0
1,2019-08-30,46.0,N,2020-01-01,Y,1745.93,1973.0,2020-01-01,2020-01-14,ZURICH AMERICAN INSURANCE CO,...,97.0,REPETITIVE MOTION,49.0,SPRAIN OR TEAR,38.0,SHOULDER(S),14569,1.0,Not Work Related,4.0
2,2019-12-06,40.0,N,2020-01-01,N,1434.80,1979.0,2020-01-01,NaN,INDEMNITY INSURANCE CO OF,...,79.0,OBJECT BEING LIFTED OR HANDLED,7.0,CONCUSSION,10.0,MULTIPLE HEAD INJURY,12589,0.0,Not Work Related,6.0
3,NaN,NaN,NaN,2020-01-01,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-12-30,61.0,N,2020-01-01,N,NaN,1958.0,2019-12-31,NaN,STATE INSURANCE FUND,...,16.0,"HAND TOOL, UTENSIL; NOT POWERED",43.0,PUNCTURE,36.0,FINGER(S),12603,0.0,Not Work Related,1.0


In [30]:
train.columns

Index(['Accident Date', 'Age at Injury', 'Alternative Dispute Resolution',
       'Assembly Date', 'Attorney/Representative', 'Average Weekly Wage',
       'Birth Year', 'C-2 Date', 'C-3 Date', 'Carrier Name', 'Carrier Type',
       'Claim Identifier', 'Claim Injury Type', 'County of Injury',
       'COVID-19 Indicator', 'District Name', 'First Hearing Date', 'Gender',
       'IME-4 Count', 'Industry Code', 'Industry Code Description',
       'Medical Fee Region', 'OIICS Nature of Injury Description',
       'WCIO Cause of Injury Code', 'WCIO Cause of Injury Description',
       'WCIO Nature of Injury Code', 'WCIO Nature of Injury Description',
       'WCIO Part Of Body Code', 'WCIO Part Of Body Description', 'Zip Code',
       'Agreement Reached', 'WCB Decision', 'Number of Dependents'],
      dtype='object')

In [23]:
sorted(train['Alternative Dispute Resolution'].unique())

['N', 'Y']

In [32]:
train[['Industry Code']]

,Industry Code
0,44.0
1,23.0
2,56.0
4,62.0
5,44.0
...,...
593451,56.0
593455,62.0
593456,62.0
593457,62.0


In [45]:
target_column = 'Claim Identifier'

target_encoder = TargetEncoder()
target_encoder.fit(train[['Industry Code Description']], train[target_column])

target_encoder.transform(train[['Industry Code Description']])

,Industry Code Description
0,5.785378e+06
1,5.778683e+06
2,5.788963e+06
4,5.756294e+06
5,5.785378e+06
...,...
593451,5.788963e+06
593455,5.756294e+06
593456,5.756294e+06
593457,5.756294e+06


In [97]:
class BinaryEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, binary_columns):
        self.binary_columns = binary_columns
        self.encodings = {}  # Store the mapping for each column

    # Fit method creates a dictionary which defines the mapping. 
    def fit(self, X, y):
        return self

    def transform(self, X):
        for col in self.binary_columns:
            unique_values = sorted(X[col].unique())

            if len(unique_values) != 2:
                raise ValueError(f"Column '{col}' does not have exactly two unique values.")

            X[col] = X[col].map({unique_values[0]: 0, unique_values[1]: 1})

        return X

class MultipleTargetEncoder(BaseEstimator, TransformerMixin):
    def __init__(self,
                # target_columns
                feature_column):
        # self.target_columns = target_columns
        self.feature_column = feature_column
        self.target_encoders = {}  # Store the target encoders for each column 


    def fit(self, X, y):
        for unique_value in y.unique():

            # for every unique value there needs to be one new column
            y_binary = (y == unique_value).astype(int)

            target_encoder = TargetEncoder(cols=[self.feature_column])
            X[self.feature_column] = X[self.feature_column].astype('category')
            
            # fit the encoder
            target_encoder.fit(X[[self.feature_column]], y_binary)

            # safe fit in a dictionary
            self.target_encoders[unique_value] = target_encoder

        return self
    
    def transform(self, X):
        
        for unique_value, encoder  in self.target_encoders.items():

            # then transform the validation set using the y_train binary target to avoid data leakage
            encoded_column = encoder.transform(X[[self.feature_column]])

            # Rename and add the encoded column to the DataFrame
            X[f'{self.feature_column}_encoded_{unique_value}'] = encoded_column

        return X
    # X contains the dataframe with the respective column that we want to encode 




In [98]:
y_train = train['Claim Injury Type']
y.unique()

array(['2. NON-COMP', '4. TEMPORARY', '3. MED ONLY', '5. PPD SCH LOSS',
       '6. PPD NSL', '1. CANCELLED', '8. DEATH', '7. PTD'], dtype=object)

In [101]:
train.columns

Index(['Accident Date', 'Age at Injury', 'Alternative Dispute Resolution',
       'Assembly Date', 'Attorney/Representative', 'Average Weekly Wage',
       'Birth Year', 'C-2 Date', 'C-3 Date', 'Carrier Name', 'Carrier Type',
       'Claim Identifier', 'Claim Injury Type', 'County of Injury',
       'COVID-19 Indicator', 'District Name', 'First Hearing Date', 'Gender',
       'IME-4 Count', 'Industry Code', 'Industry Code Description',
       'Medical Fee Region', 'OIICS Nature of Injury Description',
       'WCIO Cause of Injury Code', 'WCIO Cause of Injury Description',
       'WCIO Nature of Injury Code', 'WCIO Nature of Injury Description',
       'WCIO Part Of Body Code', 'WCIO Part Of Body Description', 'Zip Code',
       'Agreement Reached', 'WCB Decision', 'Number of Dependents',
       'Industry Code_encoded_2. NON-COMP',
       'Industry Code_encoded_4. TEMPORARY',
       'Industry Code_encoded_3. MED ONLY',
       'Industry Code_encoded_5. PPD SCH LOSS',
       'Industry Cod

In [105]:
binary_columns_list = ['Alternative Dispute Resolution', 'Attorney/Representative', 'COVID-19 Indicator']
target_encoder_list = ['Industry Code',
                    'WCIO Cause of Injury Code',
                    'WCIO Nature of Injury Code',
                    'WCIO Part Of Body Code']


preprocessing_pipeline = Pipeline(steps=[
    ('binary_encoder_bin', BinaryEncoder(binary_columns=binary_columns_list)), # Apply the binary  transformation
    ('target_encoder_in', MultipleTargetEncoder(feature_column = 'Industry Code')),
    ('target_encoder_coi', MultipleTargetEncoder(feature_column = 'WCIO Cause of Injury Code')),
    ('target_encoder_noi', MultipleTargetEncoder(feature_column = 'WCIO Nature of Injury Code')),
    ('target_encoder_pob', MultipleTargetEncoder(feature_column = 'WCIO Part Of Body Code'))
    ])

preprocessed_data = preprocessing_pipeline.fit_transform(train, y= y_train)

In [106]:
y = train['Claim Injury Type']
y.unique()

array(['2. NON-COMP', '4. TEMPORARY', '3. MED ONLY', '5. PPD SCH LOSS',
       '6. PPD NSL', '1. CANCELLED', '8. DEATH', '7. PTD'], dtype=object)

In [107]:
(y == '2. NON-COMP').astype(int)

0         1
1         0
2         0
4         1
5         0
         ..
593451    1
593455    1
593456    0
593457    1
593467    1
Name: Claim Injury Type, Length: 574021, dtype: int64

In [81]:
class MultipleTargetEncoder(BaseEstimator, TransformerMixin):
    def __init__(self,
                # target_columns
                feature_column):
        # self.target_columns = target_columns
        self.feature_column = feature_column
        self.target_encoders = {}  # Store the target encoders for each column 


    def fit(self, X, y):
        for unique_value in y.unique():

            # for every unique value there needs to be one new column
            y_binary = (y == unique_value).astype(int)

            target_encoder = TargetEncoder(cols=[self.feature_column])
            X[self.feature_column] = X[self.feature_column].astype('category')
            
            # fit the encoder
            target_encoder.fit(X[[self.feature_column]], y_binary)

            # safe fit in a dictionary
            self.target_encoders[unique_value] = target_encoder

        return self
    
    def transform(self, X, y):
        
        for key, value in self.target_encoders.items():

            # then transform the validation set using the y_train binary target to avoid data leakage
            encoded_column = value.transform(X[[self.feature_column]])

            # Rename and add the encoded column to the DataFrame
            X[f'{self.feature_column}_encoded_{key}'] = encoded_column

        return X
    # X contains the dataframe with the respective column that we want to encode 


In [82]:
mte = MultipleTargetEncoder(feature_column='Industry Code')

mte.fit(train, y)

mte.transform(train, y)


# 




,Accident Date,Age at Injury,Alternative Dispute Resolution,Assembly Date,Attorney/Representative,Average Weekly Wage,Birth Year,C-2 Date,C-3 Date,Carrier Name,...,WCB Decision,Number of Dependents,Industry Code_encoded_2. NON-COMP,Industry Code_encoded_4. TEMPORARY,Industry Code_encoded_3. MED ONLY,Industry Code_encoded_5. PPD SCH LOSS,Industry Code_encoded_6. PPD NSL,Industry Code_encoded_1. CANCELLED,Industry Code_encoded_8. DEATH,Industry Code_encoded_7. PTD
0,2019-12-30,31.0,0,2020-01-01,0,0.00,1988.0,2019-12-31,NaN,NEW HAMPSHIRE INSURANCE CO,...,Not Work Related,1.0,0.635252,0.200114,0.104658,0.045805,0.004143,0.009477,0.000435,0.000114
1,2019-08-30,46.0,0,2020-01-01,1,1745.93,1973.0,2020-01-01,2020-01-14,ZURICH AMERICAN INSURANCE CO,...,Not Work Related,4.0,0.360268,0.399553,0.114459,0.068766,0.008673,0.045499,0.002362,0.000421
2,2019-12-06,40.0,0,2020-01-01,0,1434.80,1979.0,2020-01-01,NaN,INDEMNITY INSURANCE CO OF,...,Not Work Related,6.0,0.466781,0.320730,0.125505,0.053883,0.007181,0.024540,0.001094,0.000285
4,2019-12-30,61.0,0,2020-01-01,0,NaN,1958.0,2019-12-31,NaN,STATE INSURANCE FUND,...,Not Work Related,1.0,0.541486,0.275208,0.110522,0.051793,0.009358,0.010757,0.000673,0.000201
5,2019-12-26,67.0,0,2020-01-01,0,0.00,1952.0,2019-12-31,NaN,INDEMNITY INS. OF N AMERICA,...,Not Work Related,5.0,0.635252,0.200114,0.104658,0.045805,0.004143,0.009477,0.000435,0.000114
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593451,2022-12-14,35.0,0,2022-12-30,0,0.00,1987.0,2022-12-30,NaN,STATE INSURANCE FUND,...,Not Work Related,4.0,0.466781,0.320730,0.125505,0.053883,0.007181,0.024540,0.001094,0.000285
593455,2022-12-15,33.0,0,2022-12-31,0,0.00,1989.0,2022-12-31,NaN,WESCO INSURANCE CO,...,Not Work Related,6.0,0.541486,0.275208,0.110522,0.051793,0.009358,0.010757,0.000673,0.000201
593456,2022-12-13,61.0,0,2022-12-31,0,991.08,1961.0,2022-12-31,NaN,SECURITY NATIONAL INSURANCE CO,...,Not Work Related,3.0,0.541486,0.275208,0.110522,0.051793,0.009358,0.010757,0.000673,0.000201
593457,2022-12-14,24.0,0,2022-12-31,0,0.00,1998.0,2022-12-31,NaN,TECHNOLOGY INSURANCE CO. INC.,...,Not Work Related,5.0,0.541486,0.275208,0.110522,0.051793,0.009358,0.010757,0.000673,0.000201


In [ ]:



def target_encode_multiclass(X_train_df, X_val_df, y_train,  feature_col, target_col):
    """
    Applies target encoding to a categorical feature for each class in a multi-class target variable.

    Parameters:
    - X_train (pd.DataFrame): The training feature set.
    - y_train (pd.DataFrame or pd.Series): The training target variable.
    - feature_col (str): The name of the categorical feature column to encode.
    - target_col (str): The name of the target variable column.

    Returns:
    - pd.DataFrame: The original training set concatenated with the encoded features.
    """


    # Ensure y_train is a DataFrame
    if isinstance(y_train, pd.Series):
        y_train = y_train.to_frame()

    # Initialize an empty DataFrame to store encoded features
    encoded_features_train = pd.DataFrame(index=X_train_df.index)
    encoded_features_val = pd.DataFrame(index=X_val_df.index)
    # Loop through each unique category in the targetf
    for outcome in y_train[target_col].unique():
        # Binary target for the current outcome
        y_binary = (y_train[target_col] == outcome).astype(int)

        # Apply target encoding for this outcome
        encoder = TargetEncoder(cols=[feature_col])

        # Fit and transform the training set,
        encoded_column_train = encoder.fit_transform(X_train_df[[feature_col]], y_binary)

        # then transform the validation set using the y_train binary target to avoid data leakage
        encoded_column_val = encoder.transform(X_val_df[[feature_col]])

        # Rename and add the encoded column to the DataFrame
        encoded_features_train[f'{feature_col}_encoded_{outcome}'] = encoded_column_train[feature_col]
        encoded_features_val[f'{feature_col}_encoded_{outcome}'] = encoded_column_val[feature_col]

    # Concatenate the encoded columns with the original training set
    X_train_encoded = pd.concat([X_train_df.reset_index(drop=True), encoded_features_train.reset_index(drop=True)], axis=1)
    X_val_encoded = pd.concat([X_val_df.reset_index(drop=True), encoded_features_val.reset_index(drop=True)], axis=1)

    # return both the training and validation set
    return X_train_encoded, X_val_encoded

In [43]:
from sklearn.base import BaseEstimator, TransformerMixin
from category_encoders import TargetEncoder
import pandas as pd

class MultiClassTargetEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, feature_col, target_col):
        """
        Initializes the MultiClassTargetEncoder.

        Parameters:
        - feature_col (str): The name of the categorical feature column to encode.
        - target_col (str): The name of the target variable column.
        """
        self.feature_col = feature_col
        self.target_col = target_col
        self.encoders = {}  # To store individual TargetEncoder instances for each class

    def fit(self, X, y):
        """
        Fits the target encoder for each class in the target variable.

        Parameters:
        - X (pd.DataFrame): Training feature set.
        - y (pd.Series or pd.DataFrame): Training target variable.

        Returns:
        - self
        """
        if isinstance(y, pd.Series):
            y = y.to_frame()

        self.classes_ = y[self.target_col].unique()

        for outcome in self.classes_:
            # Create a binary target for the current outcome
            y_binary = (y[self.target_col] == outcome).astype(int)

            # Initialize and fit a TargetEncoder for this outcome
            encoder = TargetEncoder(cols=[self.feature_col])
            encoder.fit(X[[self.feature_col]], y_binary)

            # Store the encoder
            self.encoders[outcome] = encoder

        return self

    def transform(self, X):
        """
        Transforms the dataset by applying the learned target encoding for each class.

        Parameters:
        - X (pd.DataFrame): Dataset to transform.

        Returns:
        - pd.DataFrame: Dataset with the encoded features added.
        """
        encoded_features = pd.DataFrame(index=X.index)

        for outcome, encoder in self.encoders.items():
            # Apply the target encoder to the feature column
            encoded_column = encoder.transform(X[[self.feature_col]])
            encoded_features[f'{self.feature_col}_encoded_{outcome}'] = encoded_column[self.feature_col]

        # Concatenate the encoded columns with the original dataset
        X_encoded = pd.concat([X.reset_index(drop=True), encoded_features.reset_index(drop=True)], axis=1)

        return X_encoded


In [46]:
train_sample = train.sample(1000, random_state=42)

MultiClassTargetEncoder('Industry Code', 'Claim Identifier').fit_transform(train_sample, train_sample)

/var/folders/nj/p20z0h9x3ssf6ll0_cdhh2yc0000gn/T/ipykernel_14007/1610782102.py:62: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  encoded_features[f'{self.feature_col}_encoded_{outcome}'] = encoded_column[self.feature_col]
/var/folders/nj/p20z0h9x3ssf6ll0_cdhh2yc0000gn/T/ipykernel_14007/1610782102.py:62: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  encoded_features[f'{self.feature_col}_encoded_{outcome}'] = encoded_column[self.feature_col]
/var/folders/nj/p20z0h9x3ssf6ll0_cdhh2yc0000gn/T/ipykernel_14007/1610782102.py:62: Perform

,Accident Date,Age at Injury,Alternative Dispute Resolution,Assembly Date,Attorney/Representative,Average Weekly Wage,Birth Year,C-2 Date,C-3 Date,Carrier Name,...,Industry Code_encoded_6107889,Industry Code_encoded_5475589,Industry Code_encoded_5426870,Industry Code_encoded_5651157,Industry Code_encoded_6073138,Industry Code_encoded_5716582,Industry Code_encoded_5716427,Industry Code_encoded_5655100,Industry Code_encoded_6070951,Industry Code_encoded_5777593
0,2021-12-30,40.0,0,2022-01-11,0,1103.58,1981.0,2022-01-11,2022-01-27,STATE INSURANCE FUND,...,5.602796e-12,4.761905e-03,5.602796e-12,5.602796e-12,5.602796e-12,5.602796e-12,4.761905e-03,4.761905e-03,5.602796e-12,5.602796e-12
1,2020-06-19,39.0,0,2020-06-29,1,974.70,1980.0,2020-06-29,NaN,STATE INSURANCE FUND,...,9.112296e-05,9.112296e-05,9.112296e-05,9.112296e-05,9.112296e-05,9.112296e-05,9.112296e-05,9.112296e-05,9.112296e-05,9.112296e-05
2,2019-12-27,38.0,0,2020-01-25,0,0.00,1981.0,2020-01-24,NaN,SAFETY NATIONAL CASUALTY CORP,...,5.602796e-12,4.761905e-03,5.602796e-12,5.602796e-12,5.602796e-12,5.602796e-12,4.761905e-03,4.761905e-03,5.602796e-12,5.602796e-12
3,2022-06-16,23.0,0,2022-06-22,0,0.00,1999.0,2022-06-22,NaN,GREAT AMERICAN ALLIANCE INS CO,...,3.543437e-04,3.543437e-04,3.543437e-04,3.543437e-04,3.543437e-04,3.543437e-04,3.543437e-04,3.543437e-04,3.543437e-04,3.543437e-04
4,2020-07-15,67.0,0,2020-07-17,0,0.00,1953.0,2020-07-17,NaN,HARTFORD CASUALTY INSURANCE CO,...,3.543437e-04,3.543437e-04,3.543437e-04,3.543437e-04,3.543437e-04,3.543437e-04,3.543437e-04,3.543437e-04,3.543437e-04,3.543437e-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2021-05-04,55.0,0,2021-05-06,0,0.00,1965.0,2021-05-05,NaN,NEW HAMPSHIRE INSURANCE CO,...,7.391541e-06,7.391541e-06,7.391541e-06,1.439302e-02,7.391541e-06,1.439302e-02,7.391541e-06,7.391541e-06,1.439302e-02,7.391541e-06
996,2021-05-05,24.0,0,2021-05-06,0,959.90,1996.0,2021-05-06,2021-10-27,STATE INSURANCE FUND,...,5.602796e-12,4.761905e-03,5.602796e-12,5.602796e-12,5.602796e-12,5.602796e-12,4.761905e-03,4.761905e-03,5.602796e-12,5.602796e-12
997,2021-01-08,57.0,0,2021-02-10,1,512.85,1963.0,2021-05-11,2021-02-04,ORISKA INSURANCE COMPANY,...,5.602796e-12,4.761905e-03,5.602796e-12,5.602796e-12,5.602796e-12,5.602796e-12,4.761905e-03,4.761905e-03,5.602796e-12,5.602796e-12
998,2022-06-03,39.0,0,2022-08-25,0,0.00,1983.0,2022-08-25,NaN,INDEMNITY INS. OF N AMERICA,...,7.391541e-06,7.391541e-06,7.391541e-06,1.439302e-02,7.391541e-06,1.439302e-02,7.391541e-06,7.391541e-06,1.439302e-02,7.391541e-06


In [ ]:

binary_columns_list = ['Alternative Dispute Resolution', 'Attorney/Representative', 'COVID-19 Indicator']

preprocessing_pipeline = Pipeline(steps=[
    ('binary_encoder', BinaryEncoder(binary_columns=binary_columns_list))  # Apply the binary  transformation
    ])

processed_data = preprocessing_pipeline.fit_transform(train)

processed_data = pd.DataFrame(processed_data, columns=train.columns)




In [ ]:
processed_data['Alternative Dispute Resolution'].unique()

array([0, 1])

In [15]:
train['Alternative Dispute Resolution'].unique()

array(['N', 'Y'], dtype=object)

In [ ]:


# drop na values and U values for alternative dispute resolution
train = train.dropna(subset=['Alternative Dispute Resolution'])
train = train[train['Alternative Dispute Resolution'] != 'U']

In [ ]:
X = train.copy()
binary_columns_list = ['Alternative Dispute Resolution']

encodings[] = {'Y': 1, 'N': 0}

for col in binary_columns_list:
    X[col] = X[col].map(encodings[col])

X['Alternative Dispute Resolution'].unique()

KeyError: 'Alternative Dispute Resolution'

In [ ]:
cat_variables_binary = ['Alternative Dispute Resolution', 'COVID-19 Indicator', 'Attorney/Representative', 'Gender']


class BinaryEncoding(BaseEstimator, TransformerMixin):
    def __init__()







import numpy as np

class CustomScaler(BaseEstimator, TransformerMixin):
    def __init__(self, feature_range=(0, 1)):
        self.feature_range = feature_range
    
    def fit(self, X, y=None):
        self.data_min_ = np.min(X, axis=0)
        self.data_range_ = np.max(X, axis=0) - self.data_min_
        self.data_range_[self.data_range_ == 0] = 1  # Avoid division by zero
        return self  # Transformer must return self according to scikit-learn convention
    
    def transform(self, X, y=None):
        X_std = (X - self.data_min_) / self.data_range_
        X_scaled = X_std * (self.feature_range[1] - self.feature_range[0]) + self.feature_range[0]
        return X_scaled
    


class TargetEncoder()
    def __init__(self, ):


    def fit():

    def transform():


    